# load bitstream

In [2]:

from pynq import Overlay
import numpy as np
from pynq import allocate
import time

ol = Overlay('top-v8.bit')

# allocate buf

In [3]:

# u4: 32-bit unsigned integers
input_buffer = allocate(shape=(25000000,), dtype='u4')
input_buffer.device_address

378535936

In [8]:
input_buffer[:16] = np.random.randint(10, size=16)
input_buffer.flush()
input_buffer[:16]

PynqBuffer([3, 7, 9, 7, 3, 3, 6, 0, 7, 9, 1, 2, 0, 3, 9, 5], dtype=uint32)

In [9]:
np.sum(input_buffer[:10])

PynqBuffer(54, dtype=uint32)

# axi-lite ctrl

In [10]:

ol.qt3_tpu_v1_0.mmio.array[1] = np.uint32(input_buffer.device_address) # DDR_BASEADDR_REG
time.sleep(0.2)

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(1) # START_REG
time.sleep(0.2)

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(0) # START_REG
time.sleep(0.2)

print("reg2: ", ol.qt3_tpu_v1_0.mmio.array[2])
for i in range(5, 10):
    print(f"reg{i}: ", ol.qt3_tpu_v1_0.mmio.array[i])

reg2:  54
reg5:  253697323
reg6:  15
reg7:  378535936
reg8:  3
reg9:  15


In [11]:
input_buffer[:25000000]

PynqBuffer([0, 8, 9, ..., 1, 6, 5], dtype=uint32)

In [10]:
input_buffer.flush()